In [ ]:
!pip install sentence_transformers pinecone-client datasets seaborn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3190f5a7dc54db6b1a7db2fc19f5a872df2c925d40b8f4962528cf6e6381d718
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully buil

In [ ]:
import torch
import os
# from transformers import TrainingArguments

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device
    device = torch.device('cuda')
    print(device)
    current_device = torch.cuda.current_device()
    num_gpus = torch.cuda.device_count()
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(current_device)
    print(num_gpus)

    print(f"PyTorch is using GPU: {gpu_name}")
else:
    # If CUDA is not available, use CPU
    device = torch.device('cpu')
    print("PyTorch is using CPU")

cuda
1
PyTorch is using GPU: NVIDIA A100-SXM4-40GB


In [ ]:
# !pip install --upgrade torch torchvision

In [ ]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";
os.environ["CUDA_LAUNCH_BLOCKING"]="1";


In [ ]:
# pip install peft


In [ ]:
# !pip install evaluate

In [ ]:
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,TrainingArguments,
    DataCollatorWithPadding

)
import evaluate
# from peft import LoftQConfig, LoraConfig, get_peft_model,PeftModel,PeftConfig

In [ ]:


class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
accuracy=evaluate.load("accuracy")
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
sentiment_data_train=pd.read_csv("/content/drive/My Drive/Clean_generated_data.csv")




In [ ]:
sentiment_data_train

,text,sentiment
0,US District Judge Charles Breyer today sentenc...,positive
1,"Slate ""Nobody wants to read editorials,"" says ...",neutral
2,Maybe pragmatism isn’t enough after all. Presi...,neutral
3,"ALBUQUERQUE, N.M. - He had traveled more than ...",positive
4,I think it’s something we need to look at. It’...,positive
...,...,...
59419,The New York Times best-selling author recentl...,negative
59420,The lucrative Outdoor Retailer show is again c...,neutral
59421,KUWAIT CITY — A planned two-day summit of Gulf...,negative
59422,The charges against David Daleiden and Sandra ...,neutral


In [ ]:
sentiment_data_train.duplicated().sum()

744

In [ ]:
sentiment_data_clean = sentiment_data_train.drop_duplicates()

In [ ]:
sentiment_data_clean

,text,sentiment
0,US District Judge Charles Breyer today sentenc...,positive
1,"Slate ""Nobody wants to read editorials,"" says ...",neutral
2,Maybe pragmatism isn’t enough after all. Presi...,neutral
3,"ALBUQUERQUE, N.M. - He had traveled more than ...",positive
4,I think it’s something we need to look at. It’...,positive
...,...,...
59419,The New York Times best-selling author recentl...,negative
59420,The lucrative Outdoor Retailer show is again c...,neutral
59421,KUWAIT CITY — A planned two-day summit of Gulf...,negative
59422,The charges against David Daleiden and Sandra ...,neutral


In [ ]:
sentiment_data_clean = sentiment_data_clean.sample(frac=1).reset_index(drop=True)

In [ ]:
sentiment_data_clean["sentiment"].value_counts()

negative    26080
neutral     21813
positive    10787
Name: sentiment, dtype: int64

In [ ]:
sentiment_data_clean.isna().sum()

text         0
sentiment    0
dtype: int64

In [ ]:
x=sentiment_data_train["text"]
y=sentiment_data_train["sentiment"]

In [ ]:
from sklearn.model_selection import train_test_split
# First, split the data into training and temporary sets
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

# Now, split the temporary set into testing and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [ ]:
X_train

293      Image source: Getty Images. Oil and gas is an ...
58038    Minutes‘ Scott Pelley ( 11/8/15) portrays Edwa...
9483     This weekend, the highly-anticipated Marvel se...
14318    COLUMBIA, S.C. (AP) - Missouri coach Cuonzo Ma...
19410    After dealing with anxiety issues, former top-...
                               ...                        
58462    Hillary Clinton, stung by an Obama mailer that...
3588     Subhankar Banerjee is coming to us from Albuqu...
1973     Authorities late Thursday morning raided Smoke...
51902    Jan 24 (Reuters) - Campbell Soup Co: * CAMPBEL...
48438    Brattin has introduced House Bill 813, making ...
Name: text, Length: 47539, dtype: object

In [ ]:
data = {'text': X_test,
        'sentiment':y_test }
df_test = pd.DataFrame(data)

In [ ]:
clean_1_test = df_test.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
clean_1_test = clean_1_test.sample(frac=1, random_state=42)
clean_1_test.dropna(subset=['text'], inplace=True)
clean_1_test

,text,sentiment
3064,By Ana Nicolaci da Costa and Charlotte Greenfi...,neutral
3485,Your tax dollars at work: US Army photoMark Th...,negative
49877,"The Plano, Texas-based company’s profit topped...",positive
47920,"While the market has done well this year, not ...",neutral
47507,The Standard &amp;amp; Poor’s CoreLogic Case-S...,neutral
...,...,...
25727,SANTA FE — Gov. Susana Martinez was in Texas o...,neutral
46976,Full speed ahead for General Dynamics' earning...,positive
25333,A planned appearance by Brian Williams on the ...,negative
5453,Senior White House officials are moving to den...,neutral


In [ ]:
x_test=clean_1_test["text"]
y_test=clean_1_test["sentiment"]

In [ ]:

model_id ="roberta-large"



# load the model from huggingface



model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=3
                                                          #  , ignore_mismatched_sizes=True
                                                           )
tokenizer = AutoTokenizer.from_pretrained(model_id,add_prefix_space=True)
# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_encoding = tokenizer(X_train.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
valid_encoding = tokenizer(X_val.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")
test_encoding = tokenizer(x_test.tolist(), padding='max_length', truncation=True, max_length=512,return_tensors="pt")


In [ ]:
y_train

293       neutral
58038    negative
9483      neutral
14318    positive
19410     neutral
           ...   
58462    negative
3588      neutral
1973     negative
51902     neutral
48438    negative
Name: sentiment, Length: 47539, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the target labels and transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)



In [ ]:
y_train.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
y_train_encoded

array([1, 0, 1, ..., 0, 1, 0])

In [ ]:
train_dataset=IMDbDataset(train_encoding,y_train_encoded)
valid_dataset=IMDbDataset(valid_encoding,y_valid_encoded)
test_dataset=IMDbDataset(test_encoding,y_test_encoded)


In [ ]:
train_dataset

In [ ]:
# !pip install accelerate -U

In [ ]:
training_args = TrainingArguments(
    # output_dir='./results',  # Fix the space before the directory path
    output_dir=model_id +"Gan",

    # num_train_epochs=1,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=64,
    # warmup_steps=500,
    # learning_rate=5e-5,
    # weight_decay=0.01,
    # logging_steps=10,
    # evaluation_strategy="epoch",
    #  logging_dir='./logs',
    # save_strategy="epoch",
    # load_best_model_at_end=True,
    # metric_for_best_model="accuracy",
    # report_to=["tensorboard"],
    # seed=42

    # save_total_limit=1


    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    # learning_rate=1e-3,
    learning_rate=2e-05,
    logging_dir='./logs',

    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # report_to=["tensorboard"],
    seed=42,
)


In [ ]:
# !pip install accelerate -U

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-7-a7e13701bd24>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.362100,0.395454,0.834595


TrainOutput(global_step=2972, training_loss=0.5147078608840824, metrics={'train_runtime': 3722.9429, 'train_samples_per_second': 12.769, 'train_steps_per_second': 0.798, 'total_flos': 4.430323477205299e+16, 'train_loss': 0.5147078608840824, 'epoch': 1.0})

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save_pretrained("output_roberta_Large_50000")

In [ ]:
import shutil

# Define source and destination paths
source_path = '/content/output_roberta_Large_50000'
destination_path = '/content/drive/My Drive/Saved_model_Just_model_roberta_Large_50000'

# Remove the destination directory if it already exists
shutil.rmtree(destination_path, ignore_errors=True)

# Copy the runs directory to Google Drive
shutil.copytree(source_path, destination_path)

'/content/drive/My Drive/Saved_model_Just_model_roberta_Large_50000'

In [ ]:
trainer.save_model()


trainer.save_state()
checkpoint_name = "test-trainer-roberta_Large_50000"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"



trainer.save_model(output_dir=final_model_path)
tokenizer.save_pretrained(final_model_path)

('./llm04_fine_tuning/test-trainer-roberta_Large_50000/tokenizer_config.json',
 './llm04_fine_tuning/test-trainer-roberta_Large_50000/special_tokens_map.json',
 './llm04_fine_tuning/test-trainer-roberta_Large_50000/vocab.json',
 './llm04_fine_tuning/test-trainer-roberta_Large_50000/merges.txt',
 './llm04_fine_tuning/test-trainer-roberta_Large_50000/added_tokens.json',
 './llm04_fine_tuning/test-trainer-roberta_Large_50000/tokenizer.json')

In [ ]:
import shutil

# Define source and destination paths
source_path = '/content/llm04_fine_tuning/test-trainer-roberta_Large_50000'
destination_path = '/content/drive/My Drive/Saved_model_all_model_roberta_roberta_Large_50000'

# Remove the destination directory if it already exists
shutil.rmtree(destination_path, ignore_errors=True)

# Copy the runs directory to Google Drive
shutil.copytree(source_path, destination_path)

'/content/drive/My Drive/Saved_model_all_model_roberta_roberta_Large_50000'

In [ ]:
trainer.save_model()


trainer.save_state()
checkpoint_name = "test-trainer-roberta_Large_50000_without_token"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"



trainer.save_model(output_dir=final_model_path)
# tokenizer.save_pretrained(final_model_path)

In [ ]:
import shutil

# Define source and destination paths
source_path = '/content/llm04_fine_tuning/test-trainer-roberta_Large_50000_without_token'
destination_path = '/content/drive/My Drive/Saved_model_all_model_roberta_roberta_Large_50000_without_token'

# Remove the destination directory if it already exists
shutil.rmtree(destination_path, ignore_errors=True)

# Copy the runs directory to Google Drive
shutil.copytree(source_path, destination_path)

'/content/drive/My Drive/Saved_model_all_model_roberta_roberta_Large_50000_without_token'

In [ ]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/Saved_model_all_model_roberta_roberta_Large_50000")


In [ ]:
inputs = tokenizer(x_test.tolist(),return_tensors="pt",padding='max_length', truncation=True, max_length=512)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
fine_tuned_model = fine_tuned_model.to(device)

Using device: cuda


In [ ]:
for key in inputs:
    inputs[key] = inputs[key].to(device)

In [ ]:
import torch.nn.functional as F
with torch.no_grad():
    #   # Move the input to the GPU
    # encoded_input = {key: value.to(device) for key, value in inputs.items()}

    # # Ensure that the model and input tensors are on the same device
    # for key in encoded_input:
    #     encoded_input[key] = encoded_input[key].to(device)

    outputs =fine_tuned_model(**inputs)

    print (outputs)
    prediction=F.softmax(outputs.logits, dim=1 )
    print (prediction)

    labels= torch.argmax(prediction, dim=1 )
    print(labels)
    classification= [fine_tuned_model.config.id2label[label_id] for label_id in labels.tolist()]

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1098,  3.5160, -3.7079],
        [ 3.5873, -0.4554, -2.7210],
        [-3.7111, -1.0984,  4.3897],
        [-1.0304,  1.7558, -0.2254],
        [ 0.8789,  1.0965, -1.2153],
        [ 3.5915,  0.7124, -3.4936],
        [-2.2186,  2.6792, -0.0220],
        [-3.6967,  1.4568,  2.2902],
        [-1.5047,  3.9119, -1.8276],
        [-0.5189,  2.8121, -1.4629],
        [ 0.3760, -0.2602, -0.0542],
        [ 1.9025,  2.7402, -3.8490],
        [ 3.4320,  0.6884, -3.2865],
        [ 4.0701, -0.6148, -2.8999],
        [ 4.3189, -0.6106, -3.2494],
        [-3.4712,  1.5470,  2.1448],
        [-2.9379, -0.1072,  2.8219],
        [-0.8564,  2.6620, -0.9092],
        [ 1.4039,  2.6975, -3.1556],
        [-3.9359,  1.2491,  2.7739],
        [ 3.4537, -0.2535, -2.7664],
        [-3.9954,  0.8478,  3.0502],
        [ 3.3688,  1.4728, -4.1323],
        [-3.6005, -0.0275,  3.6766],
        [-3.8425, -0.3821,  4.1549],
        [-3.7030,  1.3702,  2.41

In [ ]:
classification

['LABEL_1',
 'LABEL_0',
 'LABEL_2',
 'LABEL_1',
 'LABEL_1',
 'LABEL_0',
 'LABEL_1',
 'LABEL_2',
 'LABEL_1',
 'LABEL_1',
 'LABEL_0',
 'LABEL_1',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_2',
 'LABEL_2',
 'LABEL_1',
 'LABEL_1',
 'LABEL_2',
 'LABEL_0',
 'LABEL_2',
 'LABEL_0',
 'LABEL_2',
 'LABEL_2',
 'LABEL_2',
 'LABEL_2',
 'LABEL_2',
 'LABEL_1',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_1',
 'LABEL_0',
 'LABEL_0',
 'LABEL_2',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_2',
 'LABEL_1',
 'LABEL_1',
 'LABEL_1',
 'LABEL_0',
 'LABEL_0',
 'LABEL_2',
 'LABEL_1',
 'LABEL_2',
 'LABEL_1',
 'LABEL_1',
 'LABEL_1',
 'LABEL_1',
 'LABEL_0',
 'LABEL_2',
 'LABEL_1',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_1',
 'LABEL_2',
 'LABEL_0',
 'LABEL_0',
 'LABEL_0',
 'LABEL_1',
 'LABEL_1',
 'LABEL_1',
 'LABEL_2',
 'LABEL_0',
 'LABEL_1',
 'LABEL_2',
 'LABEL_0',
 'LABEL_2',
 'LABEL_2',
 'LABEL_1',
 'LABEL_0',
 'LABEL_2',
 'LABEL_1',
 'LABEL_0',
 'LABEL_1',
 'LABEL_2',
 'LABEL_0',
 'LABEL_1',
 'LA

In [ ]:
label_mapping = {"LABEL_1": "neutral", "LABEL_0":"negative", "LABEL_2": "positive"}

classification_nnn= [label_mapping[label_id] for label_id in classification]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test, classification_nnn))

              precision    recall  f1-score   support

    negative       0.90      0.86      0.88        50
     neutral       0.77      0.86      0.81        50
    positive       0.91      0.84      0.87        50

    accuracy                           0.85       150
   macro avg       0.86      0.85      0.85       150
weighted avg       0.86      0.85      0.85       150

